# P2

In [1]:
from libsvm.svmutil import *
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
import numpy as np
from sklearn.svm import SVC

## (1)SVM Accuracy and Cross-Validation for Model Selection

In [2]:
#load data 
label_train, data_train = svm_read_problem(
    'D:/course/csds600_ye/CSDS600_HW4_DogsVsCats/DogsVsCats/DogsVsCats/DogsVsCats.train')
label_test, data_test = svm_read_problem(
    'D:/course/csds600_ye/CSDS600_HW4_DogsVsCats/DogsVsCats/DogsVsCats/DogsVsCats.test')

In [3]:
label_train = np.asarray(label_train)
data_train = np.asarray(data_train)
label_test = np.asarray(label_test)
data_test = np.asarray(data_test)

In [4]:
print(np.shape(data_train))
print(np.shape(data_test))

(12500,)
(12500,)


In [5]:
#k-fold, use StratifiedKFold to make suree the sub_sets of data follow same distribution
skf = StratifiedKFold(n_splits=10, shuffle=True)

### linear kernal svm 10-fold cv

In [6]:
accuracy = 0
for train_index, val_index in skf.split(data_train,label_train):
    label_train_loop, data_train_loop = label_train[train_index].tolist(), data_train[train_index].tolist()
    label_test_loop, data_test_loop = label_train[val_index].tolist(), data_train[val_index].tolist()
    linear_kernal = svm_train(label_train_loop, data_train_loop, '-s 0 -t 0')
    #-t 0 linear kernal 
    label, acc, val = svm_predict(label_test_loop, data_test_loop, linear_kernal)
    accuracy += acc[0]
accuracy /= 10
print(f'validation accuracy : {accuracy}')

Accuracy = 61.52% (769/1250) (classification)
Accuracy = 58% (725/1250) (classification)
Accuracy = 59.84% (748/1250) (classification)
Accuracy = 59.68% (746/1250) (classification)
Accuracy = 59.44% (743/1250) (classification)
Accuracy = 58.32% (729/1250) (classification)
Accuracy = 61.12% (764/1250) (classification)
Accuracy = 60.72% (759/1250) (classification)
Accuracy = 60.24% (753/1250) (classification)
Accuracy = 59.76% (747/1250) (classification)
validation accuracy : 59.864


### all data train linear svm 

In [7]:
linear_kernal = svm_train(label_train, data_train, '-s 0 -t 0')
print('Trainng...')
label, acc, val = svm_predict(label_train, data_train, linear_kernal)
print('Testing...')
label, acc, val = svm_predict(label_test, data_test, linear_kernal)

Trainng...
Accuracy = 60.12% (7515/12500) (classification)
Testing...
Accuracy = 59.2% (7400/12500) (classification)


### polynomial kernel  svm 10-fold cv

In [8]:
accuracy = 0
for train_index, val_index in skf.split(data_train,label_train):
    label_train_loop, data_train_loop = label_train[train_index].tolist(), data_train[train_index].tolist()
    label_test_loop, data_test_loop = label_train[val_index].tolist(), data_train[val_index].tolist()
    linear_kernal = svm_train(label_train_loop, data_train_loop, '-s 0 -t 1 -d 5')
    #poly kernal with d=5
    label, acc, val = svm_predict(label_test_loop, data_test_loop, linear_kernal)
    accuracy += acc[0]
accuracy /= 10
print(f'validation accuracy: {accuracy}')

Accuracy = 50.08% (626/1250) (classification)
Accuracy = 49.92% (624/1250) (classification)
Accuracy = 50.08% (626/1250) (classification)
Accuracy = 50% (625/1250) (classification)
Accuracy = 50% (625/1250) (classification)
Accuracy = 50.08% (626/1250) (classification)
Accuracy = 50.08% (626/1250) (classification)
Accuracy = 50.08% (626/1250) (classification)
Accuracy = 50% (625/1250) (classification)
Accuracy = 49.92% (624/1250) (classification)
validation accuracy: 50.024


###  all data train polynomial kernel svm 

In [9]:
poly_kernal = svm_train(label_train, data_train, '-s 0 -t 1 -d 5')
print('Training...')
label, acc, val = svm_predict(label_train, data_train, poly_kernal)
print('Testing...')
label, acc, val = svm_predict(label_test, data_test, poly_kernal)

Training...
Accuracy = 50.024% (6253/12500) (classification)
Testing...
Accuracy = 50.048% (6256/12500) (classification)


### discussion
From the results, we could observe that for the problem, liner kernal is better than polynimial kernal with dimension 5. Poly kernal basiclly is random guessing. 

## (2)Boosting SVM 

In [10]:
def liv_svm2sk_data(data):
    data_sk = []
    for i in data:
        temp = []
        for j in range(1, 65):
            temp.append(i[j])
        data_sk.append(temp)
    data_sk = np.asarray(data_sk)     
    return data_sk

In [11]:
data_train_sk=liv_svm2sk_data(data_train)
data_test_sk=liv_svm2sk_data(data_test)

## AdaBoost Linear SVM
since linear SVM has better performence 

In [12]:
svm_l = SVC(kernel='linear')
svm_l.fit(data_train_sk, label_train)
svm_l.score(data_test_sk, label_test)

0.59184

In [13]:
clf = AdaBoostClassifier(base_estimator=svm_l, n_estimators=10, algorithm='SAMME')

In [14]:
clf.fit(data_train_sk, label_train)

AdaBoostClassifier(algorithm='SAMME', base_estimator=SVC(kernel='linear'),
                   n_estimators=10)

In [15]:
clf.score(data_test_sk, label_test)

0.55072

### discussion
The result is very interesting, the adaboost did't improve the accuracy, it turn worse actually. There are some possiable reasons at that point
1)the K=10 is too small, the adaboost svm did't converage
2)using adaboost in SVM can't improve the accuracy for reason.

## (3)Increasing the number of boosting iterations

In [16]:
clf = AdaBoostClassifier(base_estimator=svm_l, n_estimators=20, algorithm='SAMME')
clf.fit(data_train_sk, label_train)
clf.score(data_test_sk, label_test)

0.55072

Increase the K did't change the accuracy, then we use the defult clssifier, the decision tree of adaboostclassifier function test it again.

In [17]:
clf = AdaBoostClassifier(n_estimators=20)
clf.fit(data_train_sk, label_train)
clf.score(data_test_sk, label_test)

0.64424

In [18]:
clf = AdaBoostClassifier(n_estimators=200)
clf.fit(data_train_sk, label_train)
clf.score(data_test_sk, label_test)

0.67184

The result is way mush better! And the accuracy of decision tree is increasing when we using a bigger K(20->200,0.64->0.67). We did't even tunning the learning rate. 

### discussion
From above, we find that increse iteration did't help improve svm's accuracy in adaboost. That basiclly means the situation in (2) is not caused by converage problem.

## (4) Conclusion
From the experments, we have:
1) for the dogs vs cats clssify probelm, liner kernal is better than d-5 polynomial kernal SVM
2) its a bad idea using adaboost to optimize the SVM classifier 
3) weak classifier(decision tree) has better performence in adaboot

From above, we got our conclution: Using SVMs as base classifiers for Adaboost is a bad idea. 

Adaboost (and similar ensemble methods) were conceived using decision trees as base classifiers (more specifically, decision stumps, i.e. DTs with a depth of only 1);  DTs are suitable for such ensembling because they are essentially unstable classifiers, which is not the case with SVMs, hence the latter are not expected to offer much when used as base classifiers. 

On top of this, SVMs are computationally much more expensive than decision trees (let alone decision stumps), which is cause  the long processing times for traing the SVM.  Unless we have a very good reason to stick to SVMs as base classifiers for adaboost(I  doubt if there is one).